In [2]:
# import necessary packages
import pandas as pd
import numpy as np
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from ast import literal_eval
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.base import clone

import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

In [3]:
# read in data 
users = pd.read_csv('data/fasting_app_users.csv')
log = pd.read_csv('data/fasting_log.csv')

# Exploratory Data Analysis

In [4]:
# look at user data
print('Shape of the data is:', users.shape)
users.head()

Shape of the data is: (85068, 6)


id type_of_fast reason_for_fasting fasting_notification  age  gender
0   0       custom      manage_weight                  NaN   22    male
1   1       custom      manage_weight                21:30   28    male
2   2       custom     medical_advice                  NaN   44    male
3   3         16_8    increase_energy                  NaN   39  female
4   4        48_hr        live_longer                12:30   27    male

In [5]:
# look at log data
print('Shape of the data is:', log.shape)
log.head()

Shape of the data is: (160968, 7)


user_id            timestamp  fasting_duration  hunger_level  \
0  64512.0  2017-02-09 00:00:10           59640.0          60.0   
1  47163.0  2017-02-09 00:01:25           72940.0         100.0   
2  44548.0  2017-02-09 00:01:50          112700.0          50.0   
3  52175.0  2017-02-09 00:02:02           54880.0          75.0   
4  71752.0  2017-02-09 00:02:34           78680.0          90.0   

  last_meal_content                                      motivations  \
0           protein                     ['other', 'social', 'other']   
1     carbohydrates  ['fit_in', 'weight_lifting', 'health', 'other']   
2          balanced                     ['weight_lifting', 'health']   
3           protein                                               []   
4              fats                                        ['other']   

  fasting_experience  
0           Positive  
1                NaN  
2           Negative  
3           Negative  
4                NaN

I will use the pandas profiling tool to get a better sense of the data, evaluate missing/abnormal values, and look at some distributions. I will pull out any useful/interesting information into seperate plots. This gives me a quick but deep glance into the data. 

In [6]:
# pandas profiling is an awesome EDA tool 
pandas_profiling.ProfileReport(users)

In [7]:
# again, pandas profiling to get a better sense of the data
pandas_profiling.ProfileReport(log)

# Data Cleaning and Feature Engineering

Not reporting an a reason for fasting may be important overall (e.g., putting little effort into the app, or perhaps feeling shame towards the reason). Here I will add a level as missing, rather than dropping the NAs. There are other approaches I can consider, but I will keep it simple for now.

In [13]:
# fill in missing data indicating that the data is missing with a category
users['reason_for_fasting'] = np.where(users.reason_for_fasting.isnull(),'missing', users.reason_for_fasting)

I have decided to create a feature for fasting notification being on (1) or off (0). This feature can be used to cluster data as it lends to the app engagement rate. It also may be important for positive outcomes. Need to figure out if users who set a fasting notification are more active.

In [8]:
# rename column name for clarity
users.rename(columns = {"fasting_notification": "fasting_notification_time"},inplace=True)

# assign a binary indicator to indicate if a fasting notification was set
users["fasting_notification?"] = np.where(users.fasting_notification_time.notnull(), 1,0)

I am creating an indicator column for whether the fasting_type was recorded. Again, this may be important for assessing a user’s engagement with the fasting app.

In [11]:
# assign a binary indicator to indicate if fasting type was recorded
users["indicate_fast_type?"] = np.where(users.type_of_fast.notnull(),1,0)

Because there is a great deal of variation in what motivations are reported for each user, I am going to engineer the number of motivations reported for each fasting log.

In [14]:
# function to get counts of a column
def getCounts(df_column,new_col_name):
    import warnings
    warnings.simplefilter('ignore')
    counts = []
    for row in df_column:
        row = literal_eval(row)
        counts.append(len(row))
    log[new_col_name] = counts

In [15]:
getCounts(log.motivations,"motivation_counts")

In [17]:
# check it worked
log.head(3)

user_id            timestamp  fasting_duration  hunger_level  \
0  64512.0  2017-02-09 00:00:10           59640.0          60.0   
1  47163.0  2017-02-09 00:01:25           72940.0         100.0   
2  44548.0  2017-02-09 00:01:50          112700.0          50.0   

  last_meal_content                                      motivations  \
0           protein                     ['other', 'social', 'other']   
1     carbohydrates  ['fit_in', 'weight_lifting', 'health', 'other']   
2          balanced                     ['weight_lifting', 'health']   

  fasting_experience  motivation_counts  
0           Positive                  3  
1                NaN                  4  
2           Negative                  2

26% of the data are missing responses. I will keep this in mind to come back to the possibility of modeling this as well, but for now I will focus on the positive and negative class. So I will drop the nans rows moving forward.

In [18]:
# drop null values for fasting experience
log = log[log.fasting_experience.notnull()]

Now I need to engineer features that captures the engagement level of each user. I need to keep in mind that each user has started using the app at a unique time, so the engagement level needs to be normalized by the time spent with the app. So here I will define engagement level by number of logs/total days on app to get the rate of logs per day.

In [19]:
# convert to datetime
log["timestamp"] = pd.to_datetime(log.timestamp)

In [21]:
# drop user ids that are null
log = log[log.user_id.notnull()]

# fix user id type
log["user_id"] = log.user_id.astype(int)

In [22]:
import math

# get first day for each user
user_min = log.groupby("user_id")["timestamp"].min().reset_index()
user_min.rename(columns = {"timestamp": "first_day"},inplace=True)

# get last day for each user
user_max = log.groupby("user_id")["timestamp"].max().reset_index()
user_max.rename(columns = {"timestamp": "last_day"},inplace=True)

# get number of days for each user
start_end_times = pd.merge(user_min, user_max, on="user_id")
start_end_times["days"] = round((start_end_times.last_day-start_end_times.first_day)/ np.timedelta64(1, 'D'))

# fix 0 days - change to 1 day
start_end_times["days"] = np.where(start_end_times.days == 0, 1, start_end_times.days)

# round up for each day (1.3 days = 2 days)
start_end_times["days"] = [math.ceil(day) for day in start_end_times.days]

# get median fast duration for each user
user_dur = log.groupby("user_id")["fasting_duration"].median().reset_index()
user_dur.rename(columns = {"fasting_duration": "avg_fast_length"},inplace=True)
start_end_times = pd.merge(start_end_times, user_dur, on="user_id")

start_end_times.head()

user_id           first_day            last_day  days  avg_fast_length
0        1 2017-02-20 16:08:01 2017-02-20 16:08:01     1          37800.0
1        2 2017-04-09 02:00:34 2017-04-09 02:00:34     1          94080.0
2        3 2017-03-12 04:54:59 2017-03-19 04:00:18     7          95480.0
3        4 2017-03-06 14:06:40 2017-04-19 04:24:23    44          74200.0
4        5 2017-04-01 03:03:02 2017-04-01 03:03:02     1         100240.0

I will calculate logs per day and average time between fasts.

In [26]:
# get fast count for each user
user_ct = log.groupby("user_id")["timestamp"].count().reset_index()
user_ct.rename(columns = {"timestamp": "fast_count"},inplace=True)

# merge with dataframe containing number of days
final = pd.merge(start_end_times,user_ct,on="user_id")

# calculate fasting log rate (i.e., ratio of fast logs to days with app)
final["fast_logs_per_day"] = round(final.fast_count/final.days,2)

# calculate average time between convos 
final["avg_time_btwn_fasts"] = round(final.days/final.fast_count,2)

# change the infs to 0s
from numpy import inf
final["fast_logs_per_day"] = [0 if num == inf else num for num in final.fast_logs_per_day]
final["avg_time_btwn_fasts"] = [0 if num == inf else num for num in final.avg_time_btwn_fasts]

final.head()

user_id           first_day            last_day  days  avg_fast_length  \
0        1 2017-02-20 16:08:01 2017-02-20 16:08:01     1          37800.0   
1        2 2017-04-09 02:00:34 2017-04-09 02:00:34     1          94080.0   
2        3 2017-03-12 04:54:59 2017-03-19 04:00:18     7          95480.0   
3        4 2017-03-06 14:06:40 2017-04-19 04:24:23    44          74200.0   
4        5 2017-04-01 03:03:02 2017-04-01 03:03:02     1         100240.0   

   fast_count  fast_logs_per_day  avg_time_btwn_fasts  
0           1               1.00                 1.00  
1           1               1.00                 1.00  
2           5               0.71                 1.40  
3           3               0.07                14.67  
4           1               1.00                 1.00

In [27]:
users = pd.merge(users,final, left_on="id", right_on="user_id")
users.head(3)

id type_of_fast reason_for_fasting fasting_notification_time  age  gender  \
0   1       custom      manage_weight                     21:30   28    male   
1   2       custom     medical_advice                       NaN   44    male   
2   3         16_8    increase_energy                       NaN   39  female   

   fasting_notification?  indicate_fast_type?  user_id           first_day  \
0                      1                    1        1 2017-02-20 16:08:01   
1                      0                    1        2 2017-04-09 02:00:34   
2                      0                    1        3 2017-03-12 04:54:59   

             last_day  days  avg_fast_length  fast_count  fast_logs_per_day  \
0 2017-02-20 16:08:01     1          37800.0           1               1.00   
1 2017-04-09 02:00:34     1          94080.0           1               1.00   
2 2017-03-19 04:00:18     7          95480.0           5               0.71   

   avg_time_btwn_fasts  
0                  1.0  
1                  1.0  
2                  1.4

I am seeing some abnormal values for fasting duration, including a negative number and values with extremely long durations. For the sake of this analysis, I’m calling everything with a duration over 7 days as an outlier and will remove them. Before I remove them, I need to ensure that these outliers are randomly distributed for positive and negative outcome or else I will introduce bias.

In [29]:
print('Median fasting duration BEFORE removing outliers')
print(log.groupby('fasting_experience').fasting_duration.median())

# remove the negative value 
log = log[log.fasting_duration > 0]

# remove rows where fasting duration is less than or equal to 7 days
log = log[log.fasting_duration <= 604800] # this is 7 days in seconds
print('Median fasting duration AFTER removing outliers')
print(log.groupby('fasting_experience').fasting_duration.median())

Median fasting duration BEFORE removing outliers
fasting_experience
Negative     80780.0
Positive    111860.0
Name: fasting_duration, dtype: float64
Median fasting duration AFTER removing outliers
fasting_experience
Negative     79520.0
Positive    109760.0
Name: fasting_duration, dtype: float64


Great! Looking at these results, I can feel confident I am not introducing bias by dropping the rows. If these results were harder to interpret, I could perform a statistical test on the medians to be sure. 

I am going to bin the ages to make visualizations easier.

In [30]:
# bin the age variable for visualization
users['age_bins'] = np.where((users.age >= 18) & (users.age <= 25), '18to25', users.age)
users['age_bins'] = np.where((users.age >= 26) & (users.age <= 35), '26to35', users.age_bins)
users['age_bins'] = np.where((users.age >= 36) & (users.age <= 45), '36to45', users.age_bins)
users['age_bins'] = np.where((users.age >= 46) & (users.age <= 55), '46to55', users.age_bins)
users['age_bins'] = np.where((users.age >= 56) & (users.age <= 65), '56to65', users.age_bins)
users['age_bins'] = np.where((users.age >= 66) & (users.age <= 75), '66to75', users.age_bins)
users['age_bins'] = np.where((users.age >= 76) & (users.age <= 85), '76to85', users.age_bins)

## turn seconds into hours!